## Inference Script

- Module imports, dataset definition and model definition are the same as the training script.

In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.utils.data import DataLoader, Dataset
from torchvision import models
from tqdm import tqdm
from matplotlib import pyplot as plt

In [17]:
class DigitSumDataset(Dataset):
    def __init__(self, image_path, label_path):
        self.images = np.load(image_path)
        self.labels = np.load(label_path)
        self.images = torch.tensor(self.images, dtype=torch.float32) / 255.0
        self.labels = torch.tensor(self.labels, dtype=torch.float32)
        self.images = self.images.unsqueeze(1) 

    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        return self.images[idx], self.labels[idx]

In [18]:
class ResNetModel(nn.Module):
    def __init__(self):
        super(ResNetModel, self).__init__()
        self.resnet = models.resnet50(weights='IMAGENET1K_V1')
        self.resnet.conv1 = nn.Conv2d(
            1, 64, kernel_size=7, stride=2, padding=3, bias=False
        )

        num_features = self.resnet.fc.in_features
        self.resnet.fc = nn.Sequential(
            torch.nn.Linear(num_features, 128),
            torch.nn.ReLU()
        )

        self.fc = nn.Sequential(
            torch.nn.Linear(128, 1)
        )

    def forward(self, x):
        x = self.resnet(x)
        x = self.fc(x)
        return x

In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_img_path = 'DL-Project/data0.npy'
train_label_path = 'DL-Project/lab0.npy'

val_img_path = 'DL-Project/data1.npy'
val_label_path = 'DL-Project/lab1.npy'

test_img_path = 'DL-Project/data2.npy'
test_label_path = 'DL-Project/lab2.npy'

train_dataset = DigitSumDataset(train_img_path, train_label_path)
val_dataset = DigitSumDataset(val_img_path, val_label_path)
test_dataset = DigitSumDataset(test_img_path, test_label_path)

In [20]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)


In [21]:
model = ResNetModel().to(device)
model.load_state_dict(torch.load("best_model.pth"))

C:\Users\Kapil\AppData\Local\Temp\ipykernel_25192\2992537999.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("best_model.pth"))


<All keys matched successfully>

#### Inference Steps

In [22]:
def evaluate(model, data_set, dataloader):
    model.eval()
    acc = 0
    for i, (images, labels) in tqdm(enumerate(dataloader)):
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        # Round the outputs to get the predicted integer values for the sum of digits
        preds = outputs.round().flatten()  # Round outputs to the nearest integer, then flatten the tensor

        # Flatten the predictions to match the shape of the labels for comparison
        acc += torch.sum(preds == labels).item()
    acc /= len(data_set)

    return acc

In [23]:
# Get train accuracy and validation accuracy
train_acc = evaluate(model, train_dataset, train_dataloader)
val_acc = evaluate(model, val_dataset, val_dataloader)
test_acc = evaluate(model, test_dataset, test_dataloader)

print(f"Train accuracy: {train_acc:.4f}")
print(f"Validation accuracy: {val_acc:.4f}")
print(f"Test accuracy: {test_acc:.4f}")

313it [01:25,  3.68it/s]
313it [01:24,  3.70it/s]
313it [01:24,  3.69it/s]

Train accuracy: 0.4562
Validation accuracy: 0.4141
Test accuracy: 0.4177
